### Load Libraries

In [1]:
import random
import os
import numpy as np
import pandas as pd
pd.options.display.max_rows = 10

import scipy.sparse as sp

In [2]:
from utilities import utils

In [3]:
from model import model_knn

### Read Data

In [4]:
# Dataframe with tracks from NDB artists present in MPD dataset

data_root = './data/'
ndb_tracks_file = os.path.join(data_root, 'ndb_mpd_tracks.compress')

In [5]:
ndb_tracks_df = pd.read_pickle(ndb_tracks_file, compression='gzip')

In [6]:
ndb_tracks_df.shape

(15600, 18)

#### Total Number of NDB Artists present in MPD dataset

In [7]:
columns = ['2016', '2017', '2018', '2019', '2021']
ndb_mpd_editions = ndb_tracks_df.groupby('artist_name')[columns].first().isin([1]).sum()

In [8]:
ndb_editions_df= pd.DataFrame({'Edition': ndb_mpd_editions.index, 'Total_artists': ndb_mpd_editions.values})

In [9]:
ndb_editions_df

,Edition,Total_artists
0,2016,35
1,2017,30
2,2018,37
3,2019,48
4,2021,47


In [10]:
# Dataframe with tracks from NDB_MPD dataset

tracks_df = pd.read_pickle('./data/tracks_ndb.compress', compression='gzip')

In [11]:
track_title = tracks_df['track_name'].map(str) + " - " + tracks_df['artist_name']

In [12]:
# Track titles Dictionary
tracks_to_tidx = pd.Series(track_title.values, index=tracks_df.tid).to_dict()

### Load kNN Model  

Initialize kNN Model using complete __rating matrix MDP_NDB__, 25 neighbors, metric='cosine'  

### Important !!!  
First time model is run, make sure to set reTrain parameter to True so that the trained model using complete dataset MPD_NDB is saved.  
  
def __init__(self, model, n_neighbors, metric, sp_playlists, reTrain=False, data_root='./data' ,model_file='kNN.pkl')  

_kNN_model = model_knn.nneighbors_model(name_kNN, n_neighbors, metric, sparse_matrix, __True__)_

In [13]:
# Define knn input variables

name_kNN = 'knn'
n_neighbors = 25
metric = 'cosine'
sparse_matrix = sp.load_npz('./data/matrix_playlistTrackRating_ndb.npz')
sp_len = sparse_matrix.shape[1]
topk = 50

In [14]:
# k Nearest Neighbors

kNN_model = model_knn.nneighbors_model(name_kNN, n_neighbors, metric, sparse_matrix)

#### Random edition year

In [15]:
editions = [0, 2016, 2017, 2018, 2019, 2021]

In [33]:
edition = random.choice(editions)
edition

2019

### Generate input random playlist

In [34]:
playlist = utils.GenerateRandomNDBPlaylist(ndb_tracks_df, edition)

In [35]:
# Get titles from input Random Playlist tracks

input_playlist = [[tracks_to_tidx[x], x] for x in playlist]

In [36]:
df_input_playlist = pd.DataFrame.from_records(input_playlist, 
                                         columns=['title', 'tid'],
                                         index=pd.RangeIndex(start=1, stop=len(input_playlist)+1, name='rank'))

In [37]:
df_input_playlist

,title,tid
rank,,
1,537 C U B A - Orishas,171253
2,Lloré - Monsieur Periné,715715
3,7th Street - Twanguero,955474
4,I Want to Break Free - Russian Red,144954
5,Piensa en frio - Ivan Ferreiro,356571
...,...,...
44,L'eau À La Bouche - Live Au Palace 2009 - Jane...,932856
45,NW5 - Madness,1013225
46,Bedshaped - Steve Lamacq Live From Roundhouse ...,898658


### Recommendations for Random Playlist

In [38]:
sp_playlist = utils.BuildSparsePlaylist(playlist, sp_len)

In [39]:
recommends_knn = kNN_model.predict_random(sp_playlist, topk, n_neighbors)

In [40]:
# Get titles from Recommended tracks using k Nearest Neighbors Model

recommends_knn_rank = [[tracks_to_tidx[x], x] for x in recommends_knn]

In [41]:
df_knn_recs = pd.DataFrame.from_records(recommends_knn_rank, 
                                         columns=['title', 'tid'],
                                         index=pd.RangeIndex(start=1, stop=len(recommends_knn)+1, name='rank'))

In [42]:
df_knn_recs

,title,tid
rank,,
1,Diamonds On the Inside - Ben Harper,9546
2,Colors - Amos Lee,3607
3,Winter - Joshua Radin,3701
4,I'd Rather Be With You [Radio Edit] - Joshua R...,7703
5,World Spins Madly On - The Weepies,7719
...,...,...
46,A Little Less Conversation - JXL Radio Edit Re...,4703
47,I Know You Want Me (Calle Ocho) - Pitbull,12900
48,Get Free - The Vines,13236


In [43]:
def GetRandomNDBPlaylistTrackSPIDs(tracksIDs, tracksDict):
    """
    Return a ramdom playlist of track SPIDs
    :param tracksIDs: list of track IDs
    :param tracksDict: tracks map from ID to SPID   
    """     
    tracks_spIDs = [tracksDict[x] for x in tracksIDs]
    return tracks_spIDs

In [44]:
# Track tid-spid Dictionary
tracks_to_spidx = pd.Series(tracks_df.spid, index=tracks_df.tid).to_dict()

In [45]:
recommendedTracks_SPIDs = GetRandomNDBPlaylistTrackSPIDs(recommends_knn, tracks_to_spidx)

### Spotify Request Authorization - user authorization  

* Recommended tracks Playlist will be save rewritting spotify user´s playlist named - NDB_Recommends  
* Reading Spotify web API credentials from settings.env hidden file

In [30]:
import json
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials

In [31]:
# Reading Spotify web API credentials from settings.env hidden file
settings_root = "./settings/"
settingsfile = os.path.join(settings_root, "settings.env")
with open(settingsfile) as f:
    env_vars = json.loads(f.read())

# Authorization flow

client_id = env_vars['SPOTIPY_CLIENT_ID']
client_secret = env_vars['SPOTIPY_CLIENT_SECRET']
redirect_uri = env_vars['SPOTIPY_REDIRECT_URI']
username = env_vars['SPOTIPY_USER']
playlistID = env_vars['PLAYLIST_ID']
scope = 'playlist-read-private playlist-modify-public'

token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username) 

In [32]:
def GetPlaylistID(username, playlist_name):
    """
    Return Spotify ID of input playlist name
    :param username: Spotify username
    :param playlist_name: name of the playlist   
    """     
    playlist_id = ''
    playlists = sp.user_playlists(username)
    for playlist in playlists['items']:  
        if playlist['name'] == playlist_name:
            playlist_id = playlist['id']
    return playlist_id

In [33]:
playlistID = GetPlaylistID(username, 'NDB_Recommends')

In [34]:
# playlistID added to settings.env, environment variable PLAYLIST_ID

playlistID

'5kh3mdIxqKQqmRRtULXftI'

### Save Recommended Playlist at a User´s Playlist  

SaveSpotifyPlaylist function save Recommendations Playlist by replacing Playlist´s Items from your environment variable playlistID

In [31]:
def SaveSpotifyPlaylist(playlistsTracksSPIDs):
    """
    Return snapshot_id if Spotify playlistID is replaced with track list
    :param playlistsTracksSPIDs: list of track SPIDs  
    """ 
    
    # Reading Spotify web API credentials from settings.env hidden file
    settings_root = "./settings/"
    settingsfile = os.path.join(settings_root, "settings.env")
    with open(settingsfile) as f:
        env_vars = json.loads(f.read())
    
    # Authorization flow

    client_id = env_vars['SPOTIPY_CLIENT_ID']
    client_secret = env_vars['SPOTIPY_CLIENT_SECRET']
    redirect_uri = env_vars['SPOTIPY_REDIRECT_URI']
    username = env_vars['SPOTIPY_USER']
    playlistID = env_vars['PLAYLIST_ID']
    scope = 'playlist-read-private playlist-modify-public'

    token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)

    if token:
        sp = spotipy.Spotify(auth=token)
    else:
        print("Can't get token for", username)  
    
    results = sp.user_playlist_replace_tracks(username, playlistID, playlistsTracksSPIDs)
    
    if results:
        return results['snapshot_id']
    else:
        return 'Error'  

In [46]:
result = SaveSpotifyPlaylist(recommendedTracks_SPIDs)

In [47]:
result

'MTA1LDI2OGMxMTg0ZTZkNWNlZWRhYmJlZTBkOGE1ZjIxNzc1MmMxYmQ1YTg='